In [ ]:
import pandas as pd

### Columns encoding

In [ ]:
df_mayor = pd.read_csv('data/data_mayoral.csv')

In [ ]:
# Gender encoding
df_mayor['gender_est_rep'] = df_mayor['gender_est_rep'].map({'M': 1, 'F': 0})
df_mayor['gender_est_dem'] = df_mayor['gender_est_dem'].map({'M': 1, 'F': 0})

In [ ]:
# State encoding
state_categories_df = pd.read_csv('data/state_categories.csv')

# Merge with df_mayor
df_mayor = df_mayor.merge(state_categories_df, on='state_abb', how="left")
df_mayor.rename(columns={"category": "state_category"}, inplace=True)
df_mayor.drop(columns=["state_abb"], inplace=True)

# Convert 'state_category' to one-hot encoding
df_mayor = pd.get_dummies(df_mayor, columns=['state_category'], prefix='state_category')
for col in df_mayor.filter(like='state_category').columns:
    df_mayor[col] = df_mayor[col].astype(int)